# Prediction Intervals

Contains prediction intervals for all volatility forecasting methods (GARCH, NoVaS)

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import kurtosis
from novas import *
from sklearn.utils import resample

In [2]:
# data read in, adjustment and conversion to returns
sp500_data = pd.read_csv("./data/sp500index.csv")
sp500_data.index = sp500_data["Date"].astype('datetime64[ns]')
sp500_data.drop(columns=['Date'],inplace=True)

# convert to returns and normalize returns
sp500_returns = sp500_data['Close'].pct_change()[1:]
sp500_returns = (sp500_returns - np.mean(sp500_returns))/np.std(sp500_returns) # normalize to have mean 0 std 1

The basic Model-free (MF) bootstrap algorithm for prediction intervals in the setting of financial returns goes as follows:

#### 1. Use simple NoVaS to obtain transformed data $\{W_{t,a}$ for $t=p+1,\dots,n \}$ that are assumed to be approximately i.i.d. Let $p, \alpha$ and $a_i$ denote the fitted NoVaS parameters.

In [3]:
n = len(sp500_returns)
p = 16

In [4]:
W_t = simple_novas(sp500_returns, p)
kurtosis(W_t, fisher=False)

3.0215729106043225

#### 2. Calculate $\widehat{g(Y_{n+1})}$ the point predictor of $g(y_{n+1})$ as the median of the set etc.

In [5]:
point_prediction = simple_novas_prediction(sp500_returns, 16)
print(point_prediction)

0.2477919538534975


#### 3. Main bootstrap loop

#### a.) Resample randomly (with replacement) the transformed variables  $\{W_{t,a}$ for $t=p+1,\dots,n \}$ to create the pseudo-data $W_{p+1}^*,\dots, W_{n-1}^*, W_{n}^*$ and $W_{n+1}^*$

In [6]:
resampled_W_t = resample(W_t, replace=True, n_samples = len(W_t) + 1)
resampled_W_t.sort_index(inplace=True)

#### b.) Let $(Y_{1}^*,\dots,Y_{p}^*)' = (Y_{1+I}^*,\dots,Y_{p+I}^*)'$ where I is generated as a discrete random variable uniform on the values $0,1,\dots,n-p$

In [7]:
from copy import deepcopy
y_star = deepcopy(sp500_returns) # ensure that the original sp500_returns data is unchanged
I = np.random.randint(0,n-p)
y_star[0:p] = y_star[0:p] + I
y_star[0:p]

Date
1950-01-04    4878.149954
1950-01-05    4877.457921
1950-01-06    4877.271373
1950-01-09    4877.576591
1950-01-10    4876.660024
1950-01-11    4877.330609
1950-01-12    4874.956948
1950-01-13    4876.406095
1950-01-16    4877.276156
1950-01-17    4877.834830
1950-01-18    4876.902681
1950-01-19    4877.087747
1950-01-20    4877.149212
1950-01-23    4877.087376
1950-01-24    4876.595705
1950-01-25    4876.224420
Name: Close, dtype: float64

#### c.) Generate the bootstrap pseudo-data $Y_{t}^*$ for $t=p+1,\dots,n$ using equation (10.17) $$ Y_{t}^* = \frac{W_{t}}{\sqrt{1-a_0W_{t}^{*2}}}\sqrt{\sum_{i=1}^{p} a_iY_{t-i}^{*2}}$$ for $t=p+1,\dots,n$

In [19]:
def generated_novas_psuedo_data(W_t,y_star, p):
    """Generates simple novas bootstrap psuedo-data for t=p+1,...,n using eq. (10.17)"""
    
    # W_t term of eq. (10.17)
    
    a0 = 1/p
    frac_num = W_t
    frac_den = np.sqrt(1+a0*(W_t**2))
    first_term = frac_num/frac_den
    
    # A_n term of eq. (10.17)
    lagged_squared_returns = pd.Series(np.zeros(len(y_star))) 
    for i in range(p):
        lagged_squared_returns = lagged_squared_returns + (y_star**2).shift(-i)
        
    An = (lagged_squared_returns/p)
    An = pd.rolling_sum(An, window=p)
    An = np.sqrt(An)
    y_star = first_term*An
    return y_star

#### d.) Based on the bootstrap data $Y_{1}^*,\dots,Y_{n}^*$ re-estimate the NoVaS transformation, then calculate the bootstrap predictor $\widehat{g(Y_{n+1}^*)}$ as the median of the set

In [36]:
temp = simple_novas(y_star[:n-1],16)
kurtosis(temp, fisher=False)
temp_predict = simple_novas_prediction(y_star[:n-1],16)
print(temp_predict)

0.25851403383684984


#### e.) Calculate the bootstrap future value $Y_{n+1}^*$

In [37]:
y_star[-1]

0.14186746104345338

#### f.) Calculate the bootstrap root: $g(Y_{n+1}^*) - \widehat{g(Y_{n+1}^*)}$

In [39]:
bootstrap_root = point_prediction - temp_predict
print(bootstrap_root)

-0.01072207998335234


#### 4. Repeat step 3 above B timesl the B bootstrap root replicates are collected in the form of an empirical distribution whose $\alpha$-quantile is denoted $q(\alpha)$

#### 5. The $(1-\alpha)100\%$ equal-tailed prediction interval for $g(Y_{n+1})$ is given by $$ [\widehat{g(Y_{n+1})} + q(\alpha/2), \widehat{g(Y_{n+1})} + q(1-\alpha/2)] $$